<a href="https://colab.research.google.com/github/CarlTeapot/Walmart-Recruiting/blob/main/model_experiment_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets
!pip install mlflow==2.2.2
!pip install dagshub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/17.6 MB ? eta -:--:--Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.11/dist-packages (from Jinja2<4,>=2.11->mlflow==2.2.2) (3.0.2)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 23.3 MB/s eta 0:0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import opendatasets as od

od.download("https://www.kaggle.com/competitions/walmart-recruiting-store-sales-forecasting")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: matearevadze
Your Kaggle Key: ··········


100%|██████████| 2.70M/2.70M [00:00<00:00, 583MB/s]


Extracting archive ./walmart-recruiting-store-sales-forecasting/walmart-recruiting-store-sales-forecasting.zip to ./walmart-recruiting-store-sales-forecasting


In [37]:
import pandas as pd
import numpy as np
from subprocess import check_output

train_data = "train.csv"
feature_data="features.csv"

dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')


train_df=pd.read_csv("/content/walmart-recruiting-store-sales-forecasting/train.csv.zip")

feature_df=pd.read_csv("/content/walmart-recruiting-store-sales-forecasting/features.csv.zip")
store_df=pd.read_csv("/content/walmart-recruiting-store-sales-forecasting/stores.csv")



train_df=pd.merge(train_df, store_df, on='Store')
train_df=pd.merge(train_df, feature_df, on=['Store','Date'])


In [43]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import mlflow
import mlflow.sklearn
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor


### ========== Custom Transformers ==========

class DateFeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        X = X.copy()
        X['Date'] = pd.to_datetime(X['Date'])
        X['Month'] = X['Date'].dt.month
        return X

class HolidayBooleanConverter(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        X = X.copy()
        X['IsHoliday_x'] = X['IsHoliday_x'].astype(int)
        return X

class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop
    def fit(self, X, y=None): return self
    def transform(self, X):
        return X.drop(columns=self.columns_to_drop)


### ========== Load and split data ==========

df = train_df.copy()
X = df.drop(columns=['Weekly_Sales'])
y = df['Weekly_Sales']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


### ========== Define Pipeline ==========

preprocessing = Pipeline(steps=[
    ('date_features', DateFeatureExtractor()),
    ('convert_holiday', HolidayBooleanConverter()),
    ('drop_columns', DropColumns(columns_to_drop=[
        'CPI', 'Fuel_Price', 'Unemployment',
        'IsHoliday_y', 'MarkDown3', 'Date'
    ])),
    ('encode', ColumnTransformer(transformers=[
        ('type_ohe', OneHotEncoder(drop='first', sparse_output=False), ['Type']),
        ('month_ohe', OneHotEncoder(drop='first', sparse_output=False), ['Month']),
    ], remainder='passthrough'))
])


model = RandomForestRegressor(
    n_estimators=1000,
    max_depth=10,
    n_jobs=-1,
    random_state=42
)

pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing),
    ('model', model)
])


### ========== Train with sample weights ==========

sample_weight = X_train['IsHoliday_x'].astype(int).replace(1, 5).replace(0, 1)

pipeline.fit(X_train, y_train, model__sample_weight=sample_weight)


/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('date_features', DateFeatureExtractor()),
                                 ('convert_holiday', HolidayBooleanConverter()),
                                 ('drop_columns',
                                  DropColumns(columns_to_drop=['CPI',
                                                               'Fuel_Price',
                                                               'Unemployment',
                                                               'IsHoliday_y',
                                                               'MarkDown3',
                                                               'Date'])),
                                 ('encode',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('type_ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False),
                                                                   ['Type']),
                                                                  ('month_ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False),
                                                                   ['Month'])]))])),
                ('model',
                 RandomForestRegressor(max_depth=10, n_estimators=1000,
                                       n_jobs=-1, random_state=42))])

In [44]:
import mlflow
import dagshub

dagshub.init(repo_owner='CarlTeapot', repo_name='Walmart-Recruiting', mlflow=True)


name = "Random Forest model for walmart sales forecasting"


with mlflow.start_run(run_name = name):
        mlflow.sklearn.log_model(pipeline, "best_model")
        mlflow.log_metric("Validation WMAE ", wmae)


Initialized MLflow to track repo "CarlTeapot/Walmart-Recruiting"

Repository CarlTeapot/Walmart-Recruiting initialized!